# Preprocessing Enron Dataset
*Convert the current csv file to a json file with a structure ready for visualisation.*

## Convert to a convenient structure 
Each line in the csv file is a (message, recipient) combination, assuming a message does not contain duplicate recipients. I want to have a hierarchical structure:
```
array of threads: [thread_1, ..., thread_n], each thread has threadId, and
array of messages: [message_1, ..., message_m], each message has messageId, subject, sender, times, and 
array of recipients: [rec_1, ..., rec_k], each recipient has email and type

[
    { 
        threadId: e94a22508dac953,
        messages: [
            {
                messageId: e94a22508dac953,
                subject: FW: LINE SM-123,
                sender: victor.lamadrid@enron.com,
                time: 2001-10-01T14:19:03-07:00,
                recipients: [
                    { email: john.hodge@enron.com, type: to },
                    { email: john.singer@enron.com, type: cc },
                    { email: scott.neal@enron.com, type: bc },
                    { email: clarissa.garcia@enron.com, type: to }
                ]
            }
        ]
    }
]
```

In [ ]:
input_file = 'enronThread2001.csv'
output_file = 'enronThread2001.json'

In [ ]:
import pandas as pd
import pprint
import json

In [ ]:
df = pd.read_csv(input_file)
df.head()

In [ ]:
def make_thread(tid, df):
    'Return data representing a thread.'
    message_groups = df[df['TID'] == tid].groupby('MID').groups
    messages = [make_message(mid, indices, df) for mid, indices in message_groups.items()]
    
    # Filter out duplicate messages: different IDs but from the same person at the same time
    # Use a set to keep track of unique combinations: sender-timestamp
    added_ids = set()
    
    def check_unique(m):
        id = make_combined_id(m)
        if id not in added_ids:
            added_ids.add(id)
            return True
        return False

    unique_messages = [m for m in messages if check_unique(m)]            
    
    return {
        'threadId': tid,
        'messages': unique_messages
    }

def make_message(mid, indices, df):
    return {
        'messageId': mid.strip(),
        'subject': df['SUBJECT'][indices[0]].strip().replace('\"', ''),
        'sender': df['FROM'][indices[0]].strip(),
        'time': df['TIMESTAMP'][indices[0]].strip(),
        'recipients': [make_recipient(idx, df) for idx in indices]
    }

def make_recipient(index, df):
    return {
        'email': df['TO'][index],
        'type': df['TYPE'][index]
    }

def make_combined_id(message):
    return message['sender'] + '-' + message['time']

Save to data files.

In [ ]:
threads = [make_thread(tid, df) for tid in df.groupby('TID').groups.keys()]
with open(output_file, 'w') as f:
    json.dump(threads, f)

## Reload the full file to do further processing

In [135]:
import json
num_threads = 1000 # The number of longest threads (in terms of number of messages) to retrieve

In [136]:
data = json.load(open('enronThread2001.json'))
data.sort(key=lambda x: -len(x['messages']))
data = data[:num_threads]

There's a problem with email of recipients. There are some emails as `nan`. Replace them with text.

In [137]:
for t in data:
    for m in t['messages']:
        for r in m['recipients']:
            if type(r['email']) == float:
                r['email'] = ''

Add message body from the database.

In [138]:
import mysql.connector

In [139]:
cnx = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='enron')
cursor = cnx.cursor()

In [ ]:
def find_message_body(sender, time):
    query = 'select body from message where sender="{}" and date="{}"'.format(sender, time)
    cursor.execute(query)
    result = cursor.fetchone()
    return result[0] if result else ''

Have no idea why do I need to create a new cursor. With only one cursor, it stops at 80th message.

In [ ]:
for i, t in enumerate(data):
    for j, m in enumerate(t['messages']):
        cnx = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='enron')
        cursor = cnx.cursor()
#         print(i, j)
        m['body'] = find_message_body(m['sender'], m['time'])

In [ ]:
with open('threads-{}.json'.format(num_threads), 'w') as f:
    json.dump(data, f)